In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir('D:/ZipFiles/PlantSeedlingsClassification/train')

In [3]:
os.listdir()

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [4]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('D:/ZipFiles/PlantSeedlingsClassification/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)

Black-grass
Charlock
Cleavers
Common Chickweed
Common wheat
Fat Hen
Loose Silky-bent
Maize
Scentless Mayweed
Shepherds Purse
Small-flowered Cranesbill
Sugar beet


In [5]:
x_train[0].shape

(128, 128, 3)

In [6]:
dum = pd.get_dummies(y_train)

In [7]:
dum.shape

(4750, 12)

In [8]:
y_train = dum

In [9]:
y_train = np.array(y_train)
x_train = np.array(x_train)

In [10]:
x_train.shape

(4750, 128, 128, 3)

In [11]:
y_train[1500:1505]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [12]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

3800
950


In [13]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [14]:
x_train2 = x_train2/255.
x_val = x_val/255.

In [15]:
print (x_train2.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [16]:
print(y_train2.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


In [17]:
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [18]:
model = Sequential()

#Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(128, 128, 3)))
#Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, (3, 3), activation='relu'))
#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2)))
#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25))

#Add a Convolutional Layer with 63 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
#Add a Convolutional Layer with 64 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(64, (3, 3), activation='relu'))
#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2)))
#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25))

#Add a Convolutional Layer with 128 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
#Add a Convolutional Layer with 128 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(128, (3, 3), activation='relu'))
#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2)))
#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25))

#Flatten the layer
model.add(Flatten())
#Add Fully Connected Layer with 500 units and activation function as 'ReLU'
model.add(Dense(500, activation='relu'))
#Add Fully Connected Layer with 100 units and activation function as 'ReLU'
model.add(Dense(100, activation='relu'))
#Apply Dropout with 0.5 probability 
model.add(Dropout(0.5))
#Add Fully Connected Layer with 12 units and activation function as 'softmax'
model.add(Dense(12, activation='softmax'))

In [19]:
from keras import optimizers
opt = optimizers.SGD(lr = 0.01, momentum = 0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 64)       

In [21]:
model.fit(x_train2, y_train2, epochs = 25, validation_data=(x_val, y_val))

Train on 3800 samples, validate on 950 samples
Epoch 1/25
3800/3800 [==============================] - 16s 4ms/step - loss: 2.4390 - accuracy: 0.1316 - val_loss: 2.4271 - val_accuracy: 0.1274
Epoch 2/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4164 - accuracy: 0.1413 - val_loss: 2.4273 - val_accuracy: 0.1253
Epoch 3/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4079 - accuracy: 0.1374 - val_loss: 2.4288 - val_accuracy: 0.1263
Epoch 4/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4167 - accuracy: 0.1405 - val_loss: 2.4308 - val_accuracy: 0.1274
Epoch 5/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4155 - accuracy: 0.1363 - val_loss: 2.4295 - val_accuracy: 0.1274
Epoch 6/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4152 - accuracy: 0.1371 - val_loss: 2.4279 - val_accuracy: 0.1274
Epoch 7/25
3800/3800 [==============================] - 13s 3ms/step - loss: 2.4081

In [21]:
model1 = Sequential()

model1.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu',input_shape=(128, 128, 3)))
model1.add(BatchNormalization(axis=3))
model1.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization(axis=3))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model1.add(BatchNormalization(axis=3))
model1.add(Conv2D(filters = 128, kernel_size = (5,5), activation ='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization(axis=3))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model1.add(BatchNormalization(axis=3))
model1.add(Conv2D(filters = 256, kernel_size = (5,5), activation ='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization(axis=3))
model1.add(Dropout(0.25))

model1.add(Flatten())

model1.add(Dense(256, activation = "relu"))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))

model1.add(Dense(60, activation = "relu"))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))

model1.add(Dense(12, activation = "softmax"))

In [22]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 124, 124, 64)      256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 120, 120, 64)      102464    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 60, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 60, 60, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 128)      

In [23]:
opt1 = optimizers.SGD(lr = 0.01, momentum = 0.9)
model1.compile(loss = "categorical_crossentropy", optimizer = opt1, metrics=["accuracy"])

In [24]:
model1.fit(x_train2, y_train2, epochs = 25, validation_data=(x_val, y_val))

Train on 3800 samples, validate on 950 samples
Epoch 1/25
3800/3800 [==============================] - 299s 79ms/step - loss: 2.3805 - accuracy: 0.2566 - val_loss: 4.6083 - val_accuracy: 0.1495
Epoch 2/25
3800/3800 [==============================] - 255s 67ms/step - loss: 1.6389 - accuracy: 0.4363 - val_loss: 3.7961 - val_accuracy: 0.2337
Epoch 3/25
3800/3800 [==============================] - 257s 68ms/step - loss: 1.4238 - accuracy: 0.5166 - val_loss: 2.3207 - val_accuracy: 0.2811
Epoch 4/25
3800/3800 [==============================] - 257s 68ms/step - loss: 1.2033 - accuracy: 0.5953 - val_loss: 1.0150 - val_accuracy: 0.6421
Epoch 5/25
3800/3800 [==============================] - 256s 67ms/step - loss: 1.0966 - accuracy: 0.6279 - val_loss: 1.2587 - val_accuracy: 0.5832
Epoch 6/25
3800/3800 [==============================] - 256s 67ms/step - loss: 0.9998 - accuracy: 0.6758 - val_loss: 1.1797 - val_accuracy: 0.6063
Epoch 7/25
3800/3800 [==============================] - 256s 67ms/step 

In [48]:
from keras import applications
model2 = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

In [49]:
for layer in model2.layers[:5]:
    layer.trainable = False

In [50]:
model2.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [51]:
x = model2.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)
x = Dense(32, activation="relu")(x)
predictions = Dense(12, activation="softmax")(x)

In [52]:
from keras.models import Model
model_final = Model(input = model2.input, output = predictions)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [53]:
opt2 = optimizers.SGD(lr = 0.002)
model_final.compile(loss = "categorical_crossentropy", optimizer = opt2, metrics=["accuracy"])

In [54]:
model_final.fit(x_train2, y_train2, epochs = 25, validation_data=(x_val, y_val))

Train on 3800 samples, validate on 950 samples
Epoch 1/25
3800/3800 [==============================] - 33s 9ms/step - loss: 2.4357 - accuracy: 0.1471 - val_loss: 2.2527 - val_accuracy: 0.2842
Epoch 2/25
3800/3800 [==============================] - 33s 9ms/step - loss: 2.0914 - accuracy: 0.2979 - val_loss: 1.8793 - val_accuracy: 0.3463
Epoch 3/25
3800/3800 [==============================] - 33s 9ms/step - loss: 1.6848 - accuracy: 0.4305 - val_loss: 1.3272 - val_accuracy: 0.5463
Epoch 4/25
3800/3800 [==============================] - 33s 9ms/step - loss: 1.3207 - accuracy: 0.5347 - val_loss: 0.9866 - val_accuracy: 0.6600
Epoch 5/25
3800/3800 [==============================] - 33s 9ms/step - loss: 1.0775 - accuracy: 0.6318 - val_loss: 1.0218 - val_accuracy: 0.6347
Epoch 6/25
3800/3800 [==============================] - 33s 9ms/step - loss: 0.8506 - accuracy: 0.7121 - val_loss: 0.7083 - val_accuracy: 0.7505
Epoch 7/25
3800/3800 [==============================] - 33s 9ms/step - loss: 0.6876

In [55]:
os.chdir('D:/ZipFiles/PlantSeedlingsClassification/test')

In [56]:
os.listdir()

['0021e90e4.png',
 '003d61042.png',
 '007b3da8b.png',
 '0086a6340.png',
 '00c47e980.png',
 '00d090cde.png',
 '00ef713a8.png',
 '01291174f.png',
 '026716f9b.png',
 '02cfeb38d.png',
 '03566743d.png',
 '03a2ee656.png',
 '03e322a29.png',
 '03ef36742.png',
 '043449b0b.png',
 '0437393b1.png',
 '04814f36d.png',
 '05341a8a6.png',
 '060450d79.png',
 '060f1dc84.png',
 '0625f063b.png',
 '063363305.png',
 '06d12f6fa.png',
 '071cb3ece.png',
 '0751c0bbc.png',
 '07e62f903.png',
 '085974290.png',
 '0885e7690.png',
 '089ad62a7.png',
 '08d591441.png',
 '0911d3dee.png',
 '099b961ec.png',
 '0a64e3e6c.png',
 '0ad9e7dfb.png',
 '0ae6668fa.png',
 '0bf7bfb05.png',
 '0c27cf05f.png',
 '0c4199daa.png',
 '0c45ace27.png',
 '0c51bf229.png',
 '0c5f6c493.png',
 '0caeda5df.png',
 '0d117d910.png',
 '0d31e6602.png',
 '0dba99002.png',
 '0e8492cb1.png',
 '0ebf8f2f4.png',
 '0ee4ad224.png',
 '0f6cbe5e8.png',
 '0fb233ad6.png',
 '115f93ecc.png',
 '116b136de.png',
 '1191ba346.png',
 '11d3f68ff.png',
 '122913909.png',
 '12625488

In [57]:
x_test=[]

for i in os.listdir():
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128))
    x_test.append(dummy)

In [58]:
x_test = np.array(x_test)

In [59]:
x_test[0].shape

(128, 128, 3)

In [60]:
x_test2 = x_test.reshape(x_test.shape[0],128,128,3)

In [61]:
x_test2 = x_test2/255.

In [62]:
y_test = model_final.predict(x_test2)
y_pred = np.argmax(y_test, axis=1) 

In [63]:
y_pred #predicted values for x_test seedlings images

array([10,  5, 11,  3, 11,  6,  3,  5,  6,  6,  5, 10, 11,  8, 11,  5,  8,
        8,  3,  9,  3, 10,  9,  5, 11,  7,  8,  8,  3, 11,  3,  9,  6,  7,
        6,  6,  6,  6,  3,  7,  0,  4,  3, 10, 11,  7,  7,  8,  7, 10,  6,
       11,  2,  6,  7,  5,  1, 10,  2,  6, 11,  2,  1,  4,  7,  5,  6,  6,
        1,  8, 10,  3,  8,  3,  5,  4,  6,  8,  8,  6, 11,  3,  3,  4,  9,
       11,  2,  8, 11,  6,  1,  6,  5,  6,  3,  6, 10,  3,  7, 10,  5,  5,
        8, 11,  5,  5,  3,  9,  2,  2,  6,  1,  3,  9,  9, 11,  1,  1, 11,
        3,  3, 11,  2,  0,  7,  4,  1,  7,  5,  3, 11, 11,  3,  3,  5,  4,
        5,  1,  6,  3,  6, 10,  9,  3,  5, 11,  4, 10, 11,  1,  8,  6,  7,
        8,  6, 10,  3,  5,  6,  2,  1,  6,  8,  8,  9,  6,  8,  4,  3,  3,
        9,  7,  9,  3,  3,  3,  6,  2,  6,  6,  6, 10,  6,  6,  8,  6,  3,
        5,  5, 10,  0,  1,  8,  4,  1,  6,  4,  3,  6,  3,  2,  6,  1,  1,
       10,  2,  5,  3,  5,  9,  5,  1,  5,  6, 10,  2,  7,  6, 10,  3,  8,
        6,  3, 10,  4,  5

In [64]:
y_labels = np.argmax(y_val, axis=1) 

In [65]:
y_val[:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [66]:
y_labels[:10]

array([10, 10,  5,  8,  6,  6,  9,  1,  1, 11], dtype=int64)

In [67]:
y_val_pred = model_final.predict(x_val)
y_val_pred = np.argmax(y_val_pred, axis=1) 

In [68]:
y_val_pred[:10]

array([10, 10,  5,  8,  6,  6,  9,  1,  1, 11], dtype=int64)

In [69]:
from sklearn import metrics
# classification report
print("Classification report for classifier :\n%s" % (metrics.classification_report(y_labels, y_val_pred)))

# confusion matrix
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_labels, y_val_pred))

Classification report for classifier :
              precision    recall  f1-score   support

           0       0.74      0.56      0.64        55
           1       0.92      0.99      0.95        90
           2       0.98      0.78      0.87        59
           3       0.96      0.93      0.95       120
           4       0.91      0.88      0.89        48
           5       0.96      0.93      0.95       104
           6       0.83      0.91      0.87       121
           7       0.88      0.98      0.93        45
           8       0.88      0.93      0.90        95
           9       0.90      0.80      0.84        44
          10       0.96      0.99      0.97        93
          11       0.86      0.93      0.89        76

   micro avg       0.90      0.90      0.90       950
   macro avg       0.90      0.88      0.89       950
weighted avg       0.90      0.90      0.90       950

Confusion matrix:
[[ 31   0   0   0   2   1  20   0   1   0   0   0]
 [  0  89   0   0   0   0